# Training 
# - Loading images 
# - Build a simple CNN model and perform training
# - Deployment of model (waiting for AWS work account to activate first)

In [22]:
# import matplotlib.pyplot as plt # not used at the moment
import PIL
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [23]:
# path variables
PROJ_PATH = os.path.abspath(os.curdir)
PATH_TRAIN_DIR = os.path.join(PROJ_PATH, 'dataset\\train\\')
print(PROJ_PATH)

d:\Work\Personal projects\teppen-deck-predict


## Set up parameters and load images to split into training and validation set

In [24]:
# load data from training folder, split 80% as training, 20% as validation
BATCH_SIZE = 64
IMG_SIZE = (160, 160)
EPOCHS = 10

train = image_dataset_from_directory(
    PATH_TRAIN_DIR, 
    labels='inferred', 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='training',
    seed=23
    )
    
validation = image_dataset_from_directory(
    PATH_TRAIN_DIR, 
    labels='inferred', 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=23
    )

num_of_classes = len(train.class_names)

# apply bufferred prefetching to improve training performance
AUTOTUNE = tf.data.AUTOTUNE
train = train.prefetch(buffer_size=AUTOTUNE)
validation = validation.prefetch(buffer_size=AUTOTUNE)

Found 1250 files belonging to 12 classes.
Using 1000 files for training.
Found 1250 files belonging to 12 classes.
Using 250 files for validation.


## Build model and start training

In [25]:
# create convolutional neural network model
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(160, 160, 3)),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(num_of_classes)
])

# compile model and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
  train,
  validation_data=validation,
  epochs=EPOCHS
)

Epoch 1/10
16/16 [==============================] - 7s 330ms/step - loss: 2.3119 - accuracy: 0.1943 - val_loss: 1.0258 - val_accuracy: 1.0000
Epoch 2/10
16/16 [==============================] - 6s 318ms/step - loss: 0.9499 - accuracy: 0.7739 - val_loss: 0.0349 - val_accuracy: 1.0000
Epoch 3/10
16/16 [==============================] - 6s 324ms/step - loss: 0.1791 - accuracy: 0.9568 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 4/10
16/16 [==============================] - 6s 313ms/step - loss: 0.0897 - accuracy: 0.9747 - val_loss: 4.2469e-04 - val_accuracy: 1.0000
Epoch 5/10
16/16 [==============================] - 6s 325ms/step - loss: 0.0935 - accuracy: 0.9714 - val_loss: 4.4965e-04 - val_accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 6s 339ms/step - loss: 0.0290 - accuracy: 0.9896 - val_loss: 1.4375e-04 - val_accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 7s 384ms/step - loss: 0.0484 - accuracy: 0.9827 - val_loss: 5.0319e-04 - val_accurac

In [26]:
score = model.evaluate(validation)
print('[validation] loss:', score[0], ' accuracy:', score[1])

4/4 [==============================] - 1s 98ms/step - loss: 4.2447e-05 - accuracy: 1.0000
[validation] loss: 4.2446950828889385e-05  accuracy: 1.0


## Test model's performance by using the original 13 images used for data augmentation

In [29]:
PATH_TRAIN_DIR = PROJ_PATH + '\\dataset\\train\\'
PATH_IMG_DIR = PROJ_PATH + '\\dataset\\images\\'

labels = []
print('Loading training dataset...')
for index, directory in enumerate(os.listdir(PATH_TRAIN_DIR)):
    labels.append(directory)
print('Completed!')

def prediction(card_path):
    img = tf.keras.preprocessing.image.load_img(
        card_path, target_size=(160, 160)
    )
    #plt.figure()
    #plt.imshow(img)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    prediction = model.predict(img_array)
    print('predicted : ' + labels[np.argmax(prediction)])

print('Performing prediction...')
for index, filename in enumerate(os.listdir(PATH_IMG_DIR)):
    print("*******************")
    print('Input image ' + filename)
    prediction(card_path = PATH_IMG_DIR + filename)
print('Completed!')

Loading training dataset...
Completed!
Performing prediction...
*******************
Input image CHUN-LI 001.png
predicted : CHUN-LI 001
*******************
Input image CHUN-LI 002.png
predicted : CHUN-LI 002
*******************
Input image CHUN-LI 003.png
predicted : CHUN-LI 003
*******************
Input image CHUN-LI 004.png
predicted : CHUN-LI 004
*******************
Input image CHUN-LI 005.png
predicted : CHUN-LI 005
*******************
Input image CHUN-LI 006.png
predicted : CHUN-LI 006
*******************
Input image CHUN-LI 007.png
predicted : CHUN-LI 007
*******************
Input image CHUN-LI 008.png
predicted : CHUN-LI 008
*******************
Input image CHUN-LI 009.png
predicted : CHUN-LI 009
*******************
Input image CHUN-LI 010.png
predicted : CHUN-LI 010
*******************
Input image CHUN-LI 011.png
predicted : CHUN-LI 011
*******************
Input image CHUN-LI 012.png
predicted : CHUN-LI 012
*******************
Input image CHUN-LI 013.png
predicted : CHUN-LI 012
